In [ ]:
# connexion a wrds 
import wrds
db = wrds.Connection(wrds_username='gabriel02', debug=True)

Loading library list...
Done


In [4]:
import pandas as pd
import numpy as np
import random

def collect_sp500_data(start_year=2003, end_year=2022, sample_size=50):

    # Étape 1 : Obtenir la liste des tickers S&P 500 (via Compustat ou CRSP)
    sp500 = db.raw_sql("""
    SELECT DISTINCT gvkey, tic
    FROM comp.names
    WHERE spcindx = 1
    AND tic IS NOT NULL
    """)

    
    tickers = sp500['tic'].dropna().unique().tolist()
    selected_tickers = random.sample(tickers, sample_size)

    # Étape 2 : Télécharger les prix quotidiens depuis CRSP
    prices = db.raw_sql(f"""
        SELECT date, permno, ticker, prc
        FROM crsp.dsf
        WHERE ticker IN ({','.join(["'" + t + "'" for t in selected_tickers])})
        AND date BETWEEN '{start_year}-01-01' AND '{end_year}-12-31'
    """)

    # Étape 3 : Télécharger le BPA (EPS) depuis Compustat
    eps = db.raw_sql(f"""
        SELECT datadate, gvkey, tic, epspx
        FROM comp.fundq
        WHERE tic IN ({','.join(["'" + t + "'" for t in selected_tickers])})
        AND datadate BETWEEN '{start_year}-01-01' AND '{end_year}-12-31'
    """)

    # Étape 4 : Calculer le ratio P/B glissant
    # Prétraitement
    prices['date'] = pd.to_datetime(prices['date'])
    eps['datadate'] = pd.to_datetime(eps['datadate'])

    # Moyenne glissante sur 12 mois du BPA
    eps = eps.sort_values(['tic', 'datadate'])
    eps['eps_rolling'] = eps.groupby('tic')['epspx'].transform(lambda x: x.rolling(window=4, min_periods=1).mean())

    # Fusion prix + BPA
    merged = pd.merge(prices, eps, left_on=['ticker', 'date'], right_on=['tic', 'datadate'], how='left')

    # Calcul du ratio P/B
    merged['pb_ratio'] = merged['prc'] / merged['eps_rolling']

    return merged

# Exemple d'utilisation
df_sp = collect_sp500_data()
print(df_sp.head())

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "spcindx" does not exist
LINE 4:     WHERE spcindx = 1
                  ^

[SQL: 
    SELECT DISTINCT gvkey, tic
    FROM comp.names
    WHERE spcindx = 1
    AND tic IS NOT NULL
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)